Model architecture-TinyVGG from:https://poloclub.github.io/cnn-explainer/

In [ ]:
import torch
import matplotlib.pyplot as plt
from PIL import Image
import torch.nn as nn
import numpy as np

from torchvision import models, transforms
from torch.autograd import Variable
import torch.nn.functional as F

from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True) #use this if your dataset is on google drive

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
class TinyVGG(nn.Module):
  def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
    super().__init__()
    self.conv_block_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,
                     stride=2)
    )
    self.conv_block_2 = nn.Sequential(
        nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features = hidden_units*32*32,
                  out_features=output_shape)
    )
  def forward(self, x:torch.Tensor):
    x = self.conv_block_1(x)
    x = self.conv_block_2(x)
    x = self.classifier(x)
    return x

torch.manual_seed(42)
model_BCE_20epochs = TinyVGG(input_shape=3,
                  hidden_units=10,
                  output_shape=1).to(device)
model_BCE_20epochs

In [ ]:
data_transform = transforms.Compose([
    transforms.Resize(size=(128,128), antialias=None), #resize image
    transforms.RandomHorizontalFlip(p=0.5), #flip the images randomly horizontally
    transforms.ToTensor(),
])
#data_transform varible will be used in the following ImageFolder code's .

In [ ]:
# Loading and transforming data using datasets.ImageFolder
train_data_cl = datasets.ImageFolder(root='add_your_training_path',
                                     transform=data_transform, #a transform for the data
                                     target_transform=None) #a transform for the label/target
val_data_cl = datasets.ImageFolder(root='add_your_validation_path',
                                   transform=data_transform)
train_data_cl, val_data_cl

In [ ]:
# Create a new instance of TinyVGG (the same class as our saved state_dict())
# Note: loading model will error if the shapes here aren't the same as the saved version
loaded_model_for_gradcam = TinyVGG(input_shape=3,
                                   hidden_units=10, # try changing this to 128 and seeing what happens
                                   output_shape=1).to(device)

# Load in the saved state_dict()
loaded_model_for_gradcam.load_state_dict(torch.load(f="path_of_your_saved_CNN_model.pth", map_location=torch.device("cpu")))

# Send model to GPU
loaded_model_for_gradcam = loaded_model_for_gradcam.to(device)

In [ ]:
loaded_model_for_gradcam

In [ ]:
loaded_model_for_gradcam.eval()

In [ ]:
torch.manual_seed(42)
# define the hooks
gradients = None
activations = None

def backward_hook(module, grad_input, grad_output):
  global gradients
  print("Backward hook running...")
  gradients = grad_output
  # print the size of the gradient
  print(f'Gradients size: {gradients[0].size()}')

def forward_hook(module, args, output):
  global activations
  print("Forward hook running...")
  activations = output
  # Print the size of the activations
  print(f'Activations size: {activations.size()}')

In [ ]:
# Find the last convolutional layer in your model
# In your TinyVGG model(the loaded model), the last convolutional layer is in conv_block_2
last_conv_layer = loaded_model_for_gradcam.conv_block_2
last_conv_layer

In [ ]:
# Register the backward hook on the last convolutional layer
backward_hook = last_conv_layer.register_full_backward_hook(backward_hook, prepend=False)
# Register the forward hook on the last convolutional layer
forward_hook = last_conv_layer.register_forward_hook(forward_hook, prepend=False)

In [ ]:
import os
import random

def select_random_image(folder_path):
    # Get a list of all files in the folder
    all_files = os.listdir(folder_path)

    # Filter the list to include only image files (you may need to adjust this based on your file types)
    image_files = [file for file in all_files if file.lower().endswith(('.png', '.jpg', '.jpeg'))]

    if not image_files:
        print("No image files found in the specified folder.")
        return None

    # Randomly select an image from the list
    #selected_image = image_files[213]
    selected_image = random.choice(image_files)

    # Construct the full path to the selected image
    image_path = os.path.join(folder_path, selected_image)

    return image_path, selected_image

def plot_image(image_path, selected_image, destination):
    # Load and plot the image
    img = plt.imread(image_path)
    imgplot = plt.imshow(img)
    plt.axis('off')  # Turn off axis labels

    plt.savefig(os.path.join(destination, selected_image)+"_OriginalPic.png", bbox_inches='tight')
    plt.show()

# Example usage:
folder_path = "path_of_test_dataset_specific_class"
random_image, selected_image = select_random_image(folder_path)
destination = "path_to_save_GradCam_output"
if random_image:
    print(f"Randomly selected image: {random_image}")
    plot_image(random_image, selected_image, destination)
else:
    print("No image selected.")

In [ ]:
image = Image.open(random_image).convert('RGB')

transf = transforms.Compose([
    transforms.Resize((128,128), antialias=None),
    transforms.ToTensor(),
  ])
img_tensor = transf(image).to(device) #stores the tensor that represents the image

In [ ]:
img_tensor

In [ ]:
# applying the above transforms on various images.
import random
def plot_transformed_images(image_path, transform, n=4, seed=42):

  """Plots a series of random images from image_paths.

    Will open n image paths from image_paths, transform them
    with transform and plot them side by side.

    Args:
        image_paths (list): List of target image paths.
        transform (PyTorch Transforms): Transforms to apply to images.
        n (int, optional): Number of images to plot. Defaults to 3.
        seed (int, optional): Random seed for the random generator. Defaults to 42.
    """
  #random.seed(seed)
  #random_image_paths = random.sample(image_path, k=n)
  #for image_path in random_image_paths:

  with Image.open(image_path) as f:
    fig, ax = plt.subplots(1,2)
    ax[0].imshow(f)
    ax[0].set_title(f"original \nsize: {f.size}")
    ax[0].axis("off")

    # Transform and plot image
    # Note: permute() will change shape of image to suit matplotlib
    # (PyTorch default is [C, H, W] but Matplotlib is [H, W, C])
    transformed_image = transform(f).permute(1,2,0)
    ax[1].imshow(transformed_image)
    ax[1].set_title(f"Tranformed \nsize: {transformed_image.shape}")
    ax[1].axis("off")

    fig.suptitle(f"class: {image_path}", fontsize=16)
plot_transformed_images(image_path = random_image,
                        transform=transf,
                        n=4)


In [ ]:
img_tensor.shape

In [ ]:
img_tensor.unsqueeze(0).shape

In [ ]:
loaded_model_for_gradcam(img_tensor.unsqueeze(0))

In [ ]:
loaded_model_for_gradcam(img_tensor.unsqueeze(0)).sum().backward()

## -> Computing Grad-CAM

In [ ]:
torch.manual_seed(42)
# pool the gradients across the channels
pooled_gradients = torch.mean(gradients[0], dim=[0, 2, 3])

In [ ]:
# weight the channels by corresponding gradients
for i in range(activations.size()[1]):
    activations[:, i, :, :] *= pooled_gradients[i]

# average the channels of the activations
heatmap = torch.mean(activations,dim=1).squeeze()

# relu on top of the heatmap
heatmap = F.relu(heatmap)

# normalize the heatmap
heatmap /= torch.max(heatmap)

# Move the heatmap tensor to the CPU before converting to NumPy otherwise it will give the following error:
#TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.**
heatmap = heatmap.cpu()

# Now you can convert the heatmap tensor to a NumPy array
heatmap_numpy = heatmap.detach().numpy()

# draw the heatmap
plt.matshow(heatmap.detach())
plt.savefig(os.path.join(destination, selected_image)+"_HeatMap.png", bbox_inches="tight")

## -> Overlapping heatap over the opened image

In [ ]:
from torchvision.transforms.functional import to_pil_image
from matplotlib import colormaps
import numpy as np
import PIL

# Define the extent based on the dimensions of the image
extent = (0,128, 128,0)  # Replace img_width and img_height with your image dimensions
# Define the extent (left, right, bottom, top)
#(0, 128, 0, 128): earlier extent values which were causing problem
# correct = (0, 128, 128,0)

# Create a figure and plot the first image
fig, ax = plt.subplots(1,2, figsize=(9,9))
ax[0].axis('off')
ax[1].axis("off") # removes the axis markers

# First plot the original image

#ax.imshow(image.resize((128,128), resample=PIL.Image.BICUBIC))

ax[0].imshow(to_pil_image(img_tensor, mode="RGB")) #- TENSOR img - does not look like original image

# Resize the heatmap to the same size as the input image and defines
# a resample algorithm for increasing image resolution
# we need heatmap.detach() because it can't be converted to numpy array while
# requiring gradients
overlay = to_pil_image(heatmap.detach(), mode='F').resize((128,128), resample=PIL.Image.BICUBIC)

# Apply any colormap you want
cmap = colormaps['nipy_spectral']
overlay = (255 * cmap(np.asarray(overlay))[:, :, :3]).astype(np.uint8)

# Plot the heatmap on the same axes,
# but with alpha < 1 (this defines the transparency of the heatmap)
ax[0].imshow(overlay, alpha=0.4, interpolation='nearest', extent=extent)
ax[1].imshow(to_pil_image(img_tensor, mode="RGB"))
# Show the plot
plt.savefig(os.path.join(destination, selected_image)+"_alpha 0.4-Explanation.png", bbox_inches="tight")
plt.show()

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(9,9))
ax[0].axis('off')
ax[1].axis("off") # removes the axis markers
ax[0].imshow(to_pil_image(img_tensor, mode="RGB")) #- TENSOR img - does not look like original image

# Plot the heatmap on the same axes,
# but with alpha < 1 (this defines the transparency of the heatmap)
ax[0].imshow(overlay, alpha=0.2, interpolation='nearest', extent=extent)
ax[1].imshow(to_pil_image(img_tensor, mode="RGB"))
# Show the plot
plt.savefig(os.path.join(destination, selected_image)+"_alpha 0.2-Explanation.png", bbox_inches="tight")
plt.show()

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(9,9))
ax[0].axis('off')
ax[1].axis("off") # removes the axis markers

ax[0].imshow(to_pil_image(img_tensor, mode="RGB")) #- TENSOR img - does not look like original image
# Plot the heatmap on the same axes,
# but with alpha < 1 (this defines the transparency of the heatmap)
ax[0].imshow(overlay, alpha=0.6, interpolation='nearest', extent=extent)
ax[1].imshow(to_pil_image(img_tensor, mode="RGB"))
# Show the plot
plt.savefig(os.path.join(destination, selected_image)+"_alpha 0.6-Explanation.png", bbox_inches="tight")
plt.show()

In [ ]:
# Remove the hooks to avoid interfering with future computations
forward_hook.remove()
backward_hook.remove()